In [1]:
import polars as pl
import polars.selectors as cs

eng_dict = pl.read_csv(
    "eng_dict.txt",
    separator="\t",
    has_header=False,
    skip_lines=18,
    infer_schema_length=10000000,
).drop("column_2")
eng_dict = eng_dict.with_columns(cs.all().str.strip_chars())
eng_dict = eng_dict.with_columns(
    cs.matches("column_([4-9]|1\\d)").str.split(" ").list.first(),
)
eng_dict.filter(pl.col("column_11").is_not_null())

column_1,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11
str,str,str,str,str,str,str,str,str,str
"""roughcast""","""rough-cast""","""V""","""V""","""V""","""N""","""N""","""V""","""V""","""V"""


In [2]:
from collections import defaultdict

# Grammar (extended to support nested PPs etc.)
grammar = {
    "S": [["NP", "VP"], ["NP", "VP", "PP"]],
    "NP": [["d", "NP3"]],
    "NP3": [["a", "NP3"], ["n"], ["n", "PP"], ["NP3", "PP"]],
    "PP": [["p", "NP2"], ["PP", "PP"]],
    "NP2": [["d", "NP3"]],
    "VP": [["v"], ["v", "PP"]],
}

# Sentence and POS tags
words = "An old man sat on the new chair in the house".split()
pos = ["d", "a", "n", "v", "p", "d", "a", "n", "p", "d", "n"]


# Earley State
class State:
    def __init__(self, lhs, rhs, dot, start):
        self.lhs = lhs
        self.rhs = rhs
        self.dot = dot
        self.start = start

    def next_symbol(self):
        return self.rhs[self.dot] if self.dot < len(self.rhs) else None

    def is_complete(self):
        return self.dot == len(self.rhs)

    def advance(self):
        return State(self.lhs, self.rhs, self.dot + 1, self.start)

    def __eq__(self, other):
        return (self.lhs, tuple(self.rhs), self.dot, self.start) == (
            other.lhs,
            tuple(other.rhs),
            other.dot,
            other.start,
        )

    def __hash__(self):
        return hash((self.lhs, tuple(self.rhs), self.dot, self.start))

    def __repr__(self):
        before = " ".join(self.rhs[: self.dot])
        after = " ".join(self.rhs[self.dot :])
        return f"{self.lhs} → {before} • {after}, {self.start}"


# Earley Parser Function
def earley_parse(words, pos, grammar, start_symbol="S"):
    n = len(words)
    chart = [set() for _ in range(n + 1)]
    chart[0].add(State(start_symbol, grammar[start_symbol][0], 0, 0))

    for i in range(n + 1):
        added = True
        while added:
            added = False
            current_items = list(chart[i])
            for state in current_items:
                if not state.is_complete():
                    sym = state.next_symbol()

                    # Predictor
                    if sym in grammar:
                        for prod in grammar[sym]:
                            new_state = State(sym, prod, 0, i)
                            if new_state not in chart[i]:
                                chart[i].add(new_state)
                                added = True

                else:
                    # Completer
                    for prev in chart[state.start].copy():
                        if prev.next_symbol() == state.lhs:
                            new_state = prev.advance()
                            if new_state not in chart[i]:
                                chart[i].add(new_state)
                                added = True

        # Scanner (terminal symbols)
        if i < n:
            for state in chart[i]:
                if state.next_symbol() == pos[i]:
                    chart[i + 1].add(state.advance())

    return chart


# Run Parser
chart = earley_parse(words, pos, grammar)

# Display Final Chart State
print("\nFinal Chart (last column):")
for state in sorted(chart[-1], key=lambda x: x.lhs):
    print(state)

# Accept/Reject Check
success = any(s.lhs == "S" and s.is_complete() and s.start == 0 for s in chart[-1])
print("\n✅ Sentence is", "ACCEPTED" if success else "REJECTED")


Final Chart (last column):
NP2 → d NP3 • , 9
NP2 → d NP3 • , 5
NP3 → n • , 10
NP3 → NP3 PP • , 7
NP3 → n PP • , 7
NP3 → a NP3 • , 6
NP3 → NP3 PP • , 6
NP3 → NP3 • PP, 7
NP3 → NP3 • PP, 10
NP3 → n • PP, 10
NP3 → NP3 • PP, 6
PP →  • PP PP, 11
PP → p NP2 • , 4
PP → PP PP • , 4
PP → PP • PP, 8
PP →  • p NP2, 11
PP → PP • PP, 4
PP → p NP2 • , 8
S → NP VP • , 0
VP → v PP • , 3

✅ Sentence is ACCEPTED


In [3]:
from collections import defaultdict, namedtuple
from nltk.tree import Tree

# Grammar (same as before)
grammar = defaultdict(list)
grammar["S"] = [["NP", "VP"], ["NP", "VP", "PP"]]
grammar["NP"] = [["d", "NP3"]]
grammar["NP3"] = [["a", "NP3"], ["n"], ["n", "PP"], ["NP3", "PP"]]
grammar["PP"] = [["p", "NP2"], ["PP", "PP"]]
grammar["NP2"] = [["d", "NP3"]]
grammar["VP"] = [["v"], ["v", "PP"]]

words = "An old man sat on the new chair in the house".split()
pos = ["d", "a", "n", "v", "p", "d", "a", "n", "p", "d", "n"]

# Earley State with backpointer
BackState = namedtuple("BackState", ["lhs", "rhs", "dot", "start", "end", "children"])


def make_key(state):
    return (state.lhs, tuple(state.rhs), state.dot, state.start)


def earley_with_tree(words, pos, grammar, start_symbol="S"):
    n = len(words)
    chart = [defaultdict(list) for _ in range(n + 1)]
    start_state = BackState(start_symbol, grammar[start_symbol][0], 0, 0, 0, [])
    chart[0][make_key(start_state)].append(start_state)

    for i in range(n + 1):
        added = True
        while added:
            added = False
            current_items = list(chart[i].values())
            for states in current_items:
                for state in states:
                    if state.dot < len(state.rhs):
                        sym = state.rhs[state.dot]

                        # Predictor
                        if sym in grammar:
                            for prod in grammar[sym]:
                                new_state = BackState(sym, prod, 0, i, i, [])
                                key = make_key(new_state)
                                if new_state not in chart[i][key]:
                                    chart[i][key].append(new_state)
                                    added = True

                    else:
                        # Completer
                        for prev_states in chart[state.start].values():
                            for prev in prev_states:
                                if (
                                    prev.dot < len(prev.rhs)
                                    and prev.rhs[prev.dot] == state.lhs
                                ):
                                    new_children = prev.children + [state]
                                    advanced = BackState(
                                        prev.lhs,
                                        prev.rhs,
                                        prev.dot + 1,
                                        prev.start,
                                        i,
                                        new_children,
                                    )
                                    key = make_key(advanced)
                                    if advanced not in chart[i][key]:
                                        chart[i][key].append(advanced)
                                        added = True

        # Scanner
        if i < n:
            for states in chart[i].values():
                for state in states:
                    if state.dot < len(state.rhs) and state.rhs[state.dot] == pos[i]:
                        scanned = BackState(
                            state.lhs,
                            state.rhs,
                            state.dot + 1,
                            state.start,
                            i + 1,
                            state.children + [words[i]],
                        )
                        key = make_key(scanned)
                        if scanned not in chart[i + 1][key]:
                            chart[i + 1][key].append(scanned)

    return chart


# Build tree recursively
def build_tree(state):
    if isinstance(state, str):
        return state
    children = [build_tree(c) for c in state.children]
    return Tree(state.lhs, children)


# Run parser
chart = earley_with_tree(words, pos, grammar)

# Extract final complete parses
final_states = []
for states in chart[-1].values():
    for state in states:
        if state.lhs == "S" and state.dot == len(state.rhs) and state.start == 0:
            final_states.append(state)

if final_states:
    print("\n✅ Sentence is ACCEPTED")
    tree = build_tree(final_states[0])
    tree.pretty_print()
else:
    print("\n❌ Sentence is REJECTED")


✅ Sentence is ACCEPTED
                         S                                     
      ___________________|_______                               
     |                           VP                            
     |            _______________|________                      
     |           |                        PP                   
     |           |            ____________|________             
     |           |           PP                    |           
     |           |    _______|___                  |            
     NP          |   |          NP2                PP          
  ___|___        |   |    _______|___           ___|___         
 |      NP3      |   |   |          NP3        |      NP2      
 |    ___|___    |   |   |        ___|____     |    ___|____    
 |   |      NP3  |   |   |       |       NP3   |   |       NP3 
 |   |       |   |   |   |       |        |    |   |        |   
 An old     man sat  on the     new     chair  in the     house

